## Setup Docker container for DB and install lib

In [ ]:
!pip install -qU langchain_postgres
# run postgres pg_vector docker with command: docker compose up


## Setup Postgres pg_vector Vector Store and function for importing documents

In [7]:
import re
import os
from app.db.VectorStore import VectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from tqdm import tqdm
from bs4 import BeautifulSoup
from app.db.EnumDocsCollection import EnumDocsCollection

def purify_HTML(html: str)->str:
    soup = BeautifulSoup(html, features="html.parser")

    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    text = soup.get_text()

    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text


def import_source_documents(folder:str, verbose:bool=True, only_php:bool = False):
    # Split documents and store in vector db
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=10000,
        chunk_overlap=200
    )

    vectordb = VectorStore.get_vector_store(folder)
    dir = os.path.join("../data/", folder)
    for root, d_names, f_names in os.walk(dir):
        if verbose:
            progress_bar = tqdm(f_names)
        else:
            progress_bar = f_names
        for file in progress_bar:
            if verbose:
                progress_bar.set_description(file)
            if not os.path.isdir(file):
                with open(os.path.join(root, file), 'rb') as f:
                    splits = []
                    if only_php and not file.endswith(".php"):
                        continue
                    if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".mp4"):
                        continue
                    try:
                        text = f.read().decode(errors='replace')
                        if file.endswith(".html"):
                            text = purify_HTML(text)
                    except Exception as e:
                        print(e)

                    text = re.sub(r'\{[\w_-]+\}', '', text)

                    for idx, chunk in enumerate(text_splitter.split_text(text)):
                        splits.append(Document(
                            page_content=chunk,
                            metadata={"source": file, "chunk_idx": idx}
                        ))
                    try:
                        vectordb.add_documents(splits)
                    except Exception as e:
                        print(e)

## Load Base Information

In [2]:
vectordb = VectorStore.get_vector_store(EnumDocsCollection.BASE_INFO.value)
vectordb.drop_tables()

2025-01-02 15:30:09.233 
  command:

    streamlit run /home/michal/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
import_source_documents(EnumDocsCollection.BASE_INFO.value)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

companyhouseWiki.txt: 100%|██████████| 2/2 [00:00<00:00,  4.57it/s]         


## Load Companyhouse Project Code

In [ ]:
import_source_documents(EnumDocsCollection.COMPANYHOUSE_PROJ_CODE.value, verbose=False, only_php=True)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

## Load Framework Documentation

In [5]:
import_source_documents(EnumDocsCollection.FRAMEWORK_DOCS.value)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

guide-tutorial-core-validators.html: 100%|██████████| 858/858 [13:56<00:00,  1.03it/s]                         


## Load Companyhouse Documentation

In [4]:
import_source_documents(EnumDocsCollection.COMPANYHOUSE_PROJ_DOCS.value)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Sitemaps_2239660052.html: 100%|██████████| 122/122 [00:24<00:00,  4.99it/s]                                                               
0it [00:00, ?it/s]
grey_arrow_down.png: 100%|██████████| 3/3 [00:00<00:00, 12.40it/s]


(psycopg.DataError) PostgreSQL text fields cannot contain NUL (0x00) bytes
[SQL: INSERT INTO langchain_pg_embedding (id, collection_id, embedding, document, cmetadata) VALUES (%(id_m0)s::VARCHAR, %(collection_id_m0)s::UUID, %(embedding_m0)s, %(document_m0)s::VARCHAR, %(cmetadata_m0)s::JSONB) ON CONFLICT (id) DO UPDATE SET embedding = excluded.embedding, document = excluded.document, cmetadata = excluded.cmetadata]
[parameters: {'id_m0': 'da3d34e7-e6f7-4f71-95a6-d7c33fedb580', 'collection_id_m0': UUID('d741f27b-d7ad-4a81-be9e-d70321f5dd96'), 'embedding_m0': '[-0.042048949748277664,-0.0626618042588234,-0.006961728911846876,0.011563229374587536,0.07049935311079025,0.05394190922379494,0.03512005880475044,-0. ... (7779 characters truncated) ... -0.015553394332528114,-0.0005527643370442092,-0.010515833273530006,-0.0378003790974617,0.02900931052863598,0.019590334966778755,-0.04052946716547012]', 'document_m0': 'GIF89a\x08\x00\x08\x00�\x00\x00\x003f����\x01\x02\x00\x00\x00!�\x04\x04\x14\x00�\x

home_page_16.png: 100%|██████████| 1/1 [00:00<00:00, 1054.11it/s]
